In [24]:
import os
import sys
sys.path.insert(0, "/home/gstupp/projects/metaproteomics/")
#BASE = os.path.join(os.path.dirname(os.path.realpath(sys.argv[0])), "..")
BASE = "/home/gstupp/projects/Wolan/wolan/Velos_AW_20170831"
DATA = os.path.join(BASE, "data")
OUT = os.path.join(BASE, "out")

from metaproteomics import utils
import pandas as pd
from itertools import chain
import re
import shelve
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from collections import Counter
from collections import defaultdict

from metaproteomics.analysis import build_loci ,functional_analysis
from metaproteomics.analysis.DBInfo import DBInfo

f = functional_analysis.Functionizer()

db_info = DBInfo("compil_mgm")
metadata = build_loci.read_metadata("metadata.csv")

In [6]:
metadata

,Neg1rep1,Neg1rep2,Neg1rep3,Nov_unenriched1rep1,Nov_unenriched1rep2,Nov_unenriched1rep3,Nov_unenriched2rep1,Nov_unenriched2rep2,Nov_unenriched2rep3,Nov_unenriched3rep1,...,Nov_unenriched3rep3,Nov_VK1rep1,Nov_VK1rep2,Nov_VK1rep3,Nov_VK3rep1,Nov_VK3rep2,Nov_VK3rep3,Nov_VK5rep1,Nov_VK5rep2,Nov_VK5rep3
biological,1,1,1,1,1,1,2,2,2,3,...,3,1,1,1,2,2,2,3,3,3
technical,1,2,3,1,2,3,1,2,3,1,...,3,1,2,3,1,2,3,1,2,3
unenriched,False,False,False,True,True,True,True,True,True,True,...,True,False,False,False,False,False,False,False,False,False
probe,False,False,False,False,False,False,False,False,False,False,...,False,True,True,True,True,True,True,True,True,True
sample_type,neg,neg,neg,unenriched,unenriched,unenriched,unenriched,unenriched,unenriched,unenriched,...,unenriched,probe,probe,probe,probe,probe,probe,probe,probe,probe
sample_type2,probe,probe,probe,unenriched,unenriched,unenriched,unenriched,unenriched,unenriched,unenriched,...,unenriched,probe,probe,probe,probe,probe,probe,probe,probe,probe
sample_name,neg1,neg1,neg1,unenriched1,unenriched1,unenriched1,unenriched2,unenriched2,unenriched2,unenriched3,...,unenriched3,vk1,vk1,vk1,vk2,vk2,vk2,vk3,vk3,vk3
path,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...,/home/gstupp/projects/Wolan/wolan/Velos_AW_201...


In [7]:
#%% Parse samples
samples = shelve.open(os.path.join(OUT,"samples.shelve"))
for sample_name, sample_info in tqdm(list(metadata.iteritems())):
    sample = build_loci.Sample(sample_name, sample_info.path, db_info, sample_info)
    samples[sample.sample_name] = sample

100%|██████████| 21/21 [03:07<00:00,  8.92s/it]


In [8]:
# Build protein clusters for each sample
protein_clusters = shelve.open(os.path.join(OUT,"protein_clusters.shelve"))
for name,sample in samples.items():
    protein_clusters[name] = sample.build_protein_clusters()

In [39]:
# generate proteins clusters across all samples
sample_pep_quant = {sample.sample_name:sample.pep_quant for sample in samples.values()}
grouped_loci = build_loci.group_across_samples(list(chain(*protein_clusters.values())), sample_pep_quant)
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci.pkl.gz"), force=True)

In [40]:
# write out matrix
df = build_loci.to_df(grouped_loci, norm=False)
df.T.to_csv(os.path.join(OUT,"df.csv"))

In [46]:
df.shape

(21, 12706)

In [55]:
grouped_loci = utils.load(os.path.join(OUT,"grouped_loci.pkl.gz"))

In [56]:
# filter loci, min spectral count of 4 in at least two samples
min_quant = 4
min_samples = 2
grouped_loci = [locus for locus in grouped_loci if len([x for x in locus.quantification.values() if x>=min_quant]) >= min_samples]
len(grouped_loci)

9382

In [57]:
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci_filt.pkl.gz"), force=True)

In [58]:
df = build_loci.to_df(grouped_loci, norm=False)
df.T.to_csv(os.path.join(OUT,"df_filt.csv"))
df.shape

(21, 9382)

In [61]:
# run functional annotation
for locus in tqdm(grouped_loci):
    locus.annotate()
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci_filt_annot.pkl.gz"), force=True)


  0%|          | 28/9382 [00:09<53:19,  2.92it/s]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 9382/9382 [1:11:10<00:00,  2.20it/s]


In [64]:
#%% mark as human/mouse and other
from metaproteomics.analysis import taxonomy
t = taxonomy.Taxonomy(host="wl-cmadmin.scripps.edu")
# human or mouse taxids and ancestors (up to the phylum chordata (7711)):
chordata = set(t.taxid_to_taxonomy(7711)['lineage'])
human = set(t.taxid_to_taxonomy(9606)['lineage']) - chordata
mouse = set(t.taxid_to_taxonomy(10090)['lineage']) - chordata
human_mouse = human | mouse
for locus in grouped_loci:
    locus.human_mouse = True if locus.lca in human_mouse else False

In [65]:
#%% Build a locus -> metadata table
for locus in grouped_loci:
    names = [x['d'] for x in locus.prot_info]
    gn=set(chain(*[re.findall("GN=([^ ]*)",name) for name in names]))
    locus.gn = ','.join(gn) if gn else ''
    locus.gn1 = list(gn)[0] if len(gn)==1 else ''
locus_df = pd.DataFrame({locus.cluster_id: {'name': locus.name, 'human_mouse': locus.human_mouse, 'lca':locus.lca, 'gn':locus.gn, 'gn1':locus.gn1} for locus in grouped_loci}).T
locus_df.to_csv(os.path.join(OUT,"locus_df.csv"))